# PANDAPROSUMER EXAMPLE: BOOSTER HEAT PUMP WITH STORAGE UNIT

Example prepared by:
--------------------
    Izak Oberčkal Pluško, Marko Keber, Katja Klinar, Tine Seljak*, Andrej Kitanovski

    Faculty of Mechanical Engineering, University of Ljubljana, Slovenia
    
    *Contact: tine.seljak@fs.uni-lj.si

DESCRIPTION:
--------------------
This example describes how to create a single booster heat pump element in the *pandaprosumer* software package and connect it to heat storage unit which is then connected to a single consumer. The user can choose pump's type, while the demand and source temperature data is read from an Excel file and stored in pandas dataframe. It includes the information about the power required by the consumer and source temperature at each time step. 

Glossary:
---------
- Network: a configuration of connected energy generators and energy consumers
- Element: a single energy generator or a single energy consumer
- Container: a pandaprosumer data structure that contains data of an individual element; each element must have its container 
- Controller: the logic of an element that defines its behaviour and its limits
- General controller: the first controller in the network that interacts with controllers of all other elements; this controller also manages external data
- Map / mapping: a connection between two elements; contains information about the what is exchanged between the two elements

Network design philosophy:
--------------------------
In pandaprosumer, a system's component is represented by a network element. Each element is assigned a *container* and its own *element controller*. A container is a structure that contains the component's configuration data (static input data), which can include information that will not change in the analysis such as size, nominal power, efficiency, etc. The behaviour of an element is governed by its *controller*. Connections between elements are defined in *maps*, which couple output parameters of one controller to the input parameter of a controller of a connected element. 
The network is managed by a *general controller* called *ConstProfileController*. This controller is connected to all element controllers and manages dynamic input data from external sources (e.g. Excel file). For each time step it distributes the dynamic input data to the relevant element controllers. 

CREATING A NETWORK:
--------------------
If we are not in pandaprosumer parent directory, we should add it to the path so that the program knows where to find the necessary functions:

In [ ]:
import sys
import os

current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
sys.path.append(parent_directory)

1 - INPUT DATA:
---------------
First let's import libraries required for data management.

In [ ]:
import pandas as pd
from pandapower.timeseries.data_sources.frame_data import DFData

Next we need to define properties of the booster heat pump which are treated as static input data, i.e. data (characteristics) that don't change during an analysis. In this case it's just heat pump's type and arbitrary name. We also define property of the heat storage, that is it's capacity in kWh.

In [ ]:
hp_type = 'water-water1'
hp_name = 'example_hp'

q_capacity_kwh=100

We define the analysis time series.

In [ ]:
start = '2020-01-01 00:00:00'
end = '2020-01-01 23:59:59'
time_resolution = 900

Now we import our demand data and transform it into an appropriate DFData object. All data of an individual element is stored in a dedicated DFData object.

In [ ]:
demand_data = pd.read_excel('example_data/test_hp_1.xlsx')
print(demand_data.head())

dur = pd.date_range(start=start, end=end, freq='15min', tz='utc')
demand_data.index = dur
demand_input = DFData(demand_data)

2 - CREATING ELEMENTS OF THE NETWORK:
--------------------------------------

In this example, the network is made up of three elements: a source, storage and a consumer. The source is represented by a single booster heat pump, storage is defined by heat storage element and the consumer is modelled by a single heat demand element.

![network configuration](figures/bhp_storage_cons_1.png)

2.1. ELEMENT 1 (SOURCE) ---> BOOSTER HEAT PUMP ELEMENT:

First we define an empty prosumer container object which in practice can be anything. In pandaprosumer, each element of the network has its own container, which is later filled with data and results.

In [ ]:
from pandaprosumer.create import create_empty_prosumer_container

hp_prosumer = create_empty_prosumer_container()

Then we define the period of the analysis using input data about the analysis of time and also timezone and period name.

In [ ]:
from pandaprosumer.create import create_period

hp_period = create_period(hp_prosumer, time_resolution, start, end, 'utc', 'default')

Now we fill the container created above (hp_prosumer). We pass into the function the empty prosumer container created above static input data and heat pump's type variable. Finally, the last static input data (name) is passed to the function.

In [ ]:
from pandaprosumer.create import create_booster_heat_pump

hp_index = create_booster_heat_pump(
        hp_prosumer,
        hp_type,
        name=hp_name)

2.2 ELEMENT 2 (STORAGE) ---> STORAGE ELEMENT:

Then we create the second element in the network, an element that represents heat storage. Only parameter that we pass to it is its capacity.

In [ ]:
from pandaprosumer.create import create_heat_storage

heat_storage_index = create_heat_storage(
    hp_prosumer,
    q_capacity_kwh=q_capacity_kwh,
)

Finnaly we create the third element in the network, an element that represents the energy demand. In this case we have dynamic heat demand that is defined in the Excel file and is managed by the general controller. Scaling parameter scales the demand appropriately.

In [ ]:
from pandaprosumer.create import create_heat_demand

heat_demand_index = create_heat_demand(hp_prosumer, scaling=1.0)

3 - CREATING CONTROLLERS OF THE ELEMENTS:
------------------------------------------

![network configuration](figures/bhp_storage_cons_2.png)

The first step in creating controllers is to define where each element controller gets the necessary data. 

3.1.1. GENERAL CONTROLLER DATA:

The general controller manages data from external sources. The input columns for this controller are the columns of the Excel file where dynamic input data is stored. The output columns are used later in the definitions of the mappings.

In [ ]:
from pandaprosumer.controller.data_model import ConstProfileControllerData

const_controller_data = ConstProfileControllerData(
    input_columns=['t_source_k', 'demand', 'mode'],
    result_columns=['t_source_c', 'demand_c', 'mode_c'],
    period_index=hp_period
)

3.1.2. HEAT PUMP CONTROLLER DATA:

In booster heap pump data controller object we add element index of the heat pump that was created when we defined heat pump and period index that was created when we defined period.

In [ ]:
from pandaprosumer.controller.data_model import BoosterHeatPumpControllerData

booster_heat_pump_controller_data = BoosterHeatPumpControllerData(
    # element_name='booster_heat_pump',
    element_index=[0],
    period_index=hp_period
)

3.1.3. HEAT STORAGE CONTROLLER DATA:

In heat storage data controller object we add element index of the storage that was created when we defined storage and period index that was created when we defined period.

In [ ]:
from pandaprosumer.controller.data_model import HeatStorageControllerData

heat_storage_controller_data = HeatStorageControllerData(
    element_name='heat_storage',
    element_index=[heat_storage_index],
    period_index=hp_period
)

3.1.3. HEAT DEMAND (CONTROLLER) CONTROLLER DATA:

Like in heat pump we also add appropirate element index to heat demand data controller object that was created when heat demand was defined and again the period index.

In [ ]:
from pandaprosumer.controller.data_model.heat_demand import HeatDemandControllerData

heat_demand_controller_data = HeatDemandControllerData(
    element_name='heat_demand',
    element_index=[0],
    period_index=hp_period
)

3.2. CREATING CONTROLLERS:

Now we can create the controllers and connect them to their respective containers and data. We also define the topology of the network by setting the elements' priority. In this example, the network has a linear configuration (the direction of energy flow goes in one direction), so only the order parameter is set:
- The *general controller* (ConstProfileController) is always the first element in the network and is the only one that reads data from external sources,
- As the source, the heat pump controller is the second element (order 1) in the network,
- Third element represents heat storage,
- The final order is the demand. 

3.2.1. GENERAL CONTROLLER:

In [ ]:
from pandaprosumer.controller import ConstProfileController

ConstProfileController(hp_prosumer,
                           const_object=const_controller_data,
                           df_data=demand_input,
                           order=0,
                           level=0);

3.2.2. HEAT PUMP CONTROLLER:

In [ ]:
from pandaprosumer.controller.models import BoosterHeatPumpController

BoosterHeatPumpController(hp_prosumer,
                               booster_heat_pump_controller_data,
                               order=1,
                               level=0);

3.2.3. HEAT STORAGE CONTROLLER:

In [ ]:
from pandaprosumer.controller.models import HeatStorageController

HeatStorageController(hp_prosumer,
                      heat_storage_controller_data,
                      order=2,
                      level=0);

3.2.4. HEAT DEMAND (CONSUMER) CONTROLLER:

In [ ]:
from pandaprosumer.controller.models import HeatDemandController

HeatDemandController(hp_prosumer,
                     heat_demand_controller_data,
                     order=3,
                     level=0);

4 - CREATING CONNECTIONS (MAPS) BETWEEN THE CONTROLLERS:
---------------------------------------------------------

![network configuration](figures/bhp_storage_cons_3.png)

For each controller we define how it is connected to other controllers. The main parameter for the map in this case is energy flow: the output energy flow of one element is linked with the input energy flow of the connected element. 

In [ ]:
from pandaprosumer.mapping import GenericMapping

4.1. CONNECTION GENERAL CONTROLLER ---> BOOSTER HEAT PUMP:

The general controller instructs the booster heat pump what demand is expected from the consumer and source temperature at each time step.

In [ ]:
GenericMapping(hp_prosumer,
                   initiator_id=0,
                   initiator_column="t_source_c",
                   responder_id=1,
                   responder_column="t_source_k",
                   order=0);
GenericMapping(hp_prosumer,
                   initiator_id=0,
                   initiator_column="mode_c",
                   responder_id=1,
                   responder_column="mode",
                   order=0);

4.2. CONNECTION BOOSTER HEAT PUMP ---> HEAT STORAGE:

Heat storage controller instructs the bosteer heat pump controller how much heat to supply.

In [ ]:
GenericMapping(hp_prosumer,
                   initiator_id=1,
                   initiator_column="q_floor",
                   responder_id=2,
                   responder_column="q_received_kw",
                   order=0);

4.3 CONNECTION HEAT STORAGE ---> HEAT DEMAND (CONSUMER):

Heat demand controller instructs the heat storage controller how much heat to supply.

In [ ]:
GenericMapping(hp_prosumer,
                   initiator_id=2,
                   initiator_column="q_delivered_kw",
                   responder_id=3,
                   responder_column="q_received_kw",
                   order=0);

4.4. CONNECTION GENERAL CONTROLLER ---> HEAT DEMAND (CONSUMER):

The general controller sends the demand controller information about the actual demand at each time step.

In [ ]:
GenericMapping(hp_prosumer,
                   initiator_id=0,
                   initiator_column="demand_c",
                   responder_id=3,
                   responder_column="q_demand_kw",
                   order=0);

5 - RUNNING THE ANALYSIS:
-----------------------

We can now run the analysis with the input data defined above. 

In [ ]:
from pandaprosumer.run_time_series import run_timeseries

run_timeseries(hp_prosumer, hp_period)

6 - PRINTING AND PLOTTING RESULTS:
----------------------------------

First, we plot the evolution of the demand from the Excel file.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
demand_data.plot(y='demand');
plt.show()

In [ ]:
print(hp_prosumer.heat_demand)

Then we list the availabel results for the heat pump element. Results are stored in the form of time series, which can be plotted on a graph. The *.time_series* command lists all input and output dataframes. 

In [ ]:
print(hp_prosumer.time_series)

Here *.data_source* lists all available dataframes that we then specify with index in *.loc[]*.

In [ ]:
print(hp_prosumer.time_series.data_source)

Before plotting we have to look at the resulting dataframe to see which quantity (column) do we want to plot. Index can be an integer or the defined name of specific component you want to look at.

In [ ]:
print(hp_prosumer.time_series.data_source.loc[0].df)

In [ ]:
res_df = hp_prosumer.time_series
res_df.name[1] = 'example storage'
storage_name = res_df.name[1]
res_df.set_index('name', inplace=True)
print(res_df.loc[hp_name].data_source.df)

Now we can plot the evolution of COP (floor) of booster heat pump.

In [ ]:
hp_prosumer.time_series.data_source.loc[hp_name].df.cop_floor.plot()
plt.show()

We can for example also plot the evolution of produced heat for floor heating.

In [ ]:
hp_prosumer.time_series.data_source.loc[hp_name].df.q_floor.plot()
plt.show()

Regarding heat storage, we can plot it's state of charge (SOC)

In [ ]:
hp_prosumer.time_series.data_source.loc[storage_name].df.soc.plot()
plt.show()

ACKNOWLEDGEMENTS:
-----------------
The authors thank Pawel Lytaev and colleagues from the University of Kassel. Support from the SenergyNets project (XXXXX, No. XXXXXX) is gratefully acknowledged. \
#TODO: DOPOLNI ZGORNJI TEKST